In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
FOLDER_PATH = 'data/'

In [3]:
class company_prerocessing():
#     def __init__()
    def open_df(folder_path):
        df = pd.read_csv(folder_path)

        return df
    
    def cis_preprocessing(df_cis, cmp_list):
        
        if '01-20' in df_cis.columns[6]:
            df_cis_1 = df_cis.iloc[:, [1, 6, 7, 8]]
        else:
            df_cis_1 = df_cis.iloc[:, [1, 7, 8,9]]
                
        df_cis_1.columns = ['', df_cis_1.columns[1], df_cis_1.columns[2], df_cis_1.columns[3]]
        df_cis_1 = df_cis_1.T
        
        
        df_cis_1 = df_cis_1.rename(columns=df_cis_1.iloc[0])
        df_cis_1 = df_cis_1.drop(df_cis_1.index[0])
        df_cis_1.drop('label_ko', inplace=True, axis =1)
        
        df_cis_1 = df_cis_1.reset_index().rename(columns={'index': 'date'})

        df_cis_1['company'] = cmp_list
        df_cis_1['date'] = df_cis_1['date'].str[9:]
        print('cis_preprocessing complete')

        # df_cis_1.to_csv('data/최종/포괄손익계산서 전처리/'+cmp_list+'.csv', index=False)

        return df_cis_1
    
        
    def bs_preprocessing(df_bs, cmp_list):
        try:
            if df_bs.columns[6].isdigit() == True:
                df_bs_1 = df_bs.iloc[:, [1, 6, 7, 8]]
            elif df_bs.columns[7].isdigit() == True:
                df_bs_1 = df_bs.iloc[:, [1, 7, 8, 9]]
            else:
                df_bs_1 = df_bs.iloc[:, [1, 8, 9, 10]]
        except:
            df_bs_1 = df_bs.iloc[:, 0:4]
            
        df_bs_1.columns = ['', df_bs_1.columns[1], df_bs_1.columns[2], df_bs_1.columns[3]]
        df_bs_1 = df_bs_1.T
        
        df_bs_1 = df_bs_1.rename(columns=df_bs_1.iloc[0])
        df_bs_1 = df_bs_1.drop(df_bs_1.index[0])
        df_bs_1.drop('label_ko', inplace=True, axis =1)
            
        df_bs_1 = df_bs_1.reset_index().rename(columns={'index': 'date'})
        
       
        df_bs_1['company'] = cmp_list
        print('bs_preprocessing complete')
        
        # df_bs_1.to_csv('data/최종/재무재표 전처리/'+cmp_list+'.csv', index=False)
        return df_bs_1

    def save_df(folder_path, df, cmp_list):
        df.to_csv(folder_path + cmp_list + '.csv', index=False)

    def total_dataframe(self, df_cis_1, df_bs_1, df_cf_1, cmp_list):
        data_total = pd.merge(df_cis_1, df_bs_1, on='date')
        data_total = pd.merge(data_total, df_cf_1, on='date')
        data_total.to_csv('data/total/' + cmp_list + ' total.csv', index=False)

        return data_total

In [4]:
# if '__ma|in__' == __name__:
var_1_dic = {'자본총계': 4, '자산총계': 4, '매입채무': 11, '매출채권': 11,
         '부채총계': 4, '유동부채': 4, '유동자산': 4, '재고자산': 4}
var_2_dic = {'매출총이익': 5, '법인세비용차감': 15, '당기순이익': 10}

bs_a = pd.DataFrame(columns=['date', 'company'])
cis_a = pd.DataFrame(columns=['date', 'company'])

file_list1 = os.listdir(FOLDER_PATH +'건실기업_재무상태표/')
file_list2 = os.listdir(FOLDER_PATH +'건실기업_포괄손익계산서/')

error = []


for cmp_list1, cmp_list2 in zip(file_list1, file_list2):
    print(cmp_list1, cmp_list2)
    # 재무 상태표
    df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + '건실기업_재무상태표/' + cmp_list1))
    df_bs_1 = company_prerocessing.bs_preprocessing(df, cmp_list1[:-13])
    company_prerocessing.save_df(os.path.join(FOLDER_PATH + '/전처리/건실기업_재무상태표/'), df_bs_1, cmp_list1[:-13])

    list1 = ['date', 'company']
    for var1, var1_ in var_1_dic.items():
        for bs in df_bs_1.columns:
            if var1 in bs:
                if var1_ >= len(bs):
                    list1.append(bs)
    print(df_bs_1[list1])
    try:
        bs_a = pd.concat([bs_a, df_bs_1[list1]])
    except:
        error.append(cmp_list1)
        pass




    df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + '건실기업_포괄손익계산서/' + cmp_list2))
    df_cis_1 = company_prerocessing.cis_preprocessing(df, cmp_list2[:-15])
    company_prerocessing.save_df(os.path.join(FOLDER_PATH + '/전처리/건실기업_포괄손익계산서/'), df_cis_1, cmp_list2[:-15])



    list1 = ['date', 'company']
    for var2, var2_ in var_2_dic.items():
        for cis in df_cis_1.columns:
            if var2 in cis:
                if var2_ >= len(cis):
                    list1.append(cis)
    print(df_cis_1[list1])
    cis_a = pd.concat([cis_a, df_cis_1[list1]])



bs_a.reset_index(inplace=True, drop=True)
cis_a.reset_index(inplace=True, drop=True)
company_prerocessing.save_df(os.path.join(FOLDER_PATH + '/전처리/건실기업_재무상태표/'), bs_a, '재무상태표 통합')
company_prerocessing.save_df(os.path.join(FOLDER_PATH + '/전처리/건실기업_포괄손익계산서/'), cis_a, '포괄손익계산서 통합')


AP시스템 재무상태표 원본.csv AP시스템 포괄손익계산서 원본.csv
bs_preprocessing complete
       date company            자본총계            자산총계          단기매입채무  \
0  20191231   AP시스템  114122896319.0  420965874474.0   80609784110.0   
1  20181231   AP시스템  106415129289.0  420986942846.0  106227160234.0   
2  20171231   AP시스템   89489340129.0  468030720270.0  124188931730.0   

             부채총계            유동부채            유동자산            재고자산  
0  306842978155.0  286807571145.0  252429049428.0  134134276231.0  
1  314571813557.0  291103024173.0  284424737407.0  141833640942.0  
2  378541380141.0  342537437568.0  324465331530.0   52439640871.0  
cis_preprocessing complete
       date company           매출총이익 법인세비용차감전순이익(손실)      당기순이익(손실)
0  20191231   AP시스템   72622039976.0   12402291609.0   9451469310.0
1  20181231   AP시스템  101171251514.0   27943908986.0  25227011908.0
2  20171231   AP시스템   62076108001.0   26316191703.0  19088064733.0
DMS 재무상태표 원본.csv DMS 포괄손익계산서 원본.csv
bs_preprocessing complete
       date company 

In [5]:
# concat error
error

['ITX엠투엠 재무상태표 원본.csv',
 '대한그린파워 재무상태표 원본.csv',
 '바이오스마트 재무상태표 원본.csv',
 '세운메디칼 재무상태표 원본.csv']

In [6]:
cis_a

,date,company,매출총이익,법인세비용차감전순이익(손실),당기순이익(손실),법인세비용차감전순이익,당기순이익,당기순이익의 귀속,법인세비용차감전순손실,당기순이익(손실).,계속영업당기순이익,중단영업당기순이익
0,20191231,AP시스템,72622039976.0,12402291609.0,9451469310.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20181231,AP시스템,101171251514.0,27943908986.0,25227011908.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20171231,AP시스템,62076108001.0,26316191703.0,19088064733.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20191231,DMS,65300086704.0,21544765028.0,17236232679.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20181231,DMS,75967702026.0,21868026243.0,15184128518.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
160,20181231,파커스,11625134287.0,4678354325.0,4523593272.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,20171231,파커스,10196061327.0,-2488130070.0,-2587245117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,20191231,휴림로봇,2566585107.0,-9519403684.0,-9519403684.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,20181231,휴림로봇,2736815004.0,-18128689566.0,-19026770757.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
cis_a.columns

Index(['date', 'company', '매출총이익', '법인세비용차감전순이익(손실)', '당기순이익(손실)',
       '법인세비용차감전순이익', '당기순이익', '당기순이익의 귀속', '법인세비용차감전순손실', '당기순이익(손실).',
       '계속영업당기순이익', '중단영업당기순이익'],
      dtype='object')

In [9]:
bs_a.columns

Index(['date', 'company', '자본총계', '자산총계', '단기매입채무', '부채총계', '유동부채', '유동자산',
       '재고자산', '장기매입채무및기타채무', '매출채권', '매입채무', '매입채무 및 미지급금', '매출채권 및 수취채권',
       '장기성매출채권', '매입채무및기타채무', '매출채권및기타채권', '매출채권및계약자산', '장기매출채권', '장기매입채무',
       '매입채무 및 기타채무', '매출채권 및 기타채권'],
      dtype='object')

In [7]:
bs_a

,date,company,자본총계,자산총계,단기매입채무,부채총계,유동부채,유동자산,재고자산,장기매입채무및기타채무,...,매입채무 및 미지급금,매출채권 및 수취채권,장기성매출채권,매입채무및기타채무,매출채권및기타채권,매출채권및계약자산,장기매출채권,장기매입채무,매입채무 및 기타채무,매출채권 및 기타채권
0,20191231,AP시스템,114122896319.0,420965874474.0,80609784110.0,306842978155.0,286807571145.0,252429049428.0,134134276231.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20181231,AP시스템,106415129289.0,420986942846.0,106227160234.0,314571813557.0,291103024173.0,284424737407.0,141833640942.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20171231,AP시스템,89489340129.0,468030720270.0,124188931730.0,378541380141.0,342537437568.0,324465331530.0,52439640871.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20191231,DMS,166005674664.0,302514623836.0,NaN,136508949172.0,135345962490.0,134517297914.0,14498465305.0,424713972.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20181231,DMS,151479426195.0,317314698057.0,NaN,165835271862.0,162426446113.0,159939971787.0,14616273960.0,408825749.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,20181231,톱텍,343847522619.0,392793260017.0,NaN,48945737398.0,43622028025.0,284796131539.0,22227254843.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,20171231,톱텍,327411093181.0,445178864294.0,NaN,117767771113.0,109316843705.0,350687949863.0,29323717595.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,20191231,파커스,82702280980.0,117947378909.0,NaN,35245097929.0,24061944253.0,65564770182.0,8556851910.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,20181231,파커스,93602193817.0,120125782119.0,NaN,26523588302.0,26523588302.0,81861160795.0,11084600428.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
pd.merge(cis_a, bs_a, on=['date','company'])

,date,company,매출총이익,법인세비용차감전순이익(손실),당기순이익(손실),자본총계,자산총계,부채총계,유동부채,유동자산,재고자산
0,20111231,금강제강,591907148.0,-13509435372.0,-11260872179.0,61279687814.0,194206670604.0,132926982790.0,113985044750.0,100969321003.0,37436698391.0
1,20101231,금강제강,13444476213.0,2073005833.0,1519307289.0,72211501426.0,179481870466.0,107270369040.0,85302556870.0,99806911954.0,33343253080.0
2,20111231,배명금속,8510337046.0,376240588.0,627781200.0,95296464290.0,208619609146.0,113323144856.0,106984768096.0,120559651206.0,66618482419.0
3,20101231,배명금속,7700935418.0,-5482424803.0,-5245282081.0,77745475250.0,179399062436.0,101653587186.0,95049265962.0,94315952312.0,55218399050.0
4,20111231,풍림산업,2072700894.0,-41366351473.0,-35852563764.0,152837431616.0,1374255960342.0,1221418528726.0,752955724323.0,919718310569.0,30002292372.0
5,20101231,풍림산업,29044164067.0,-91069640922.0,-74624076085.0,185149189773.0,1188801152279.0,1003651962506.0,954388334556.0,771447482160.0,34818379461.0


In [7]:
file_list1

['AP시스템 재무상태표 원본.csv',
 'DMS 재무상태표 원본.csv',
 'HRS 재무상태표 원본.csv',
 'ITX엠투엠 재무상태표 원본.csv',
 'SGA 재무상태표 원본.csv',
 'SM Life Design 재무상태표 원본.csv',
 '노바렉스 재무상태표 원본.csv',
 '대성엘텍 재무상태표 원본.csv',
 '대성파인텍 재무상태표 원본.csv',
 '대한과학 재무상태표 원본.csv',
 '대한그린파워 재무상태표 원본.csv',
 '동국산업 재무상태표 원본.csv',
 '로보스타 재무상태표 원본.csv',
 '로체시스템즈 재무상태표 원본.csv',
 '메디톡스 재무상태표 원본.csv',
 '멜파스 재무상태표 원본.csv',
 '명성티엔에스 재무상태표 원본.csv',
 '미래생명자원 재무상태표 원본.csv',
 '미코 재무상태표 원본.csv',
 '바른테크놀로지 재무상태표 원본.csv',
 '바이오스마트 재무상태표 원본.csv',
 '베셀 재무상태표 원본.csv',
 '비아트론 재무상태표 원본.csv',
 '삼강엠앤티 재무상태표 원본.csv',
 '상상인인더스트리 재무상태표 원본.csv',
 '서플러스글로벌 재무상태표 원본.csv',
 '성도이엔지 재무상태표 원본.csv',
 '세경하이테크 재무상태표 원본.csv',
 '세운메디칼 재무상태표 원본.csv',
 '슈펙스비앤피 재무상태표 원본.csv',
 '스킨앤스킨 재무상태표 원본.csv',
 '아바코 재무상태표 원본.csv',
 '아스트 재무상태표 원본.csv',
 '아이씨디 재무상태표 원본.csv',
 '아이원스 재무상태표 원본.csv',
 '에스에프에이 재무상태표 원본.csv',
 '에이프런티어 재무상태표 원본.csv',
 '엔피케이 재무상태표 원본.csv',
 '엘오티베큠 재무상태표 원본.csv',
 '예스티 재무상태표 원본.csv',
 '오르비텍 재무상태표 원본.csv',
 '와이엔텍 재무상태표 원본.csv',
 '와이엠티 재무상태표 원본.csv',
 '우리조명 재무상태표 원본.cs

In [11]:
df_bs = company_prerocessing.open_df(os.path.join(FOLDER_PATH + '건실기업_재무상태표/' + file_list1[3]))

In [12]:
df_bs.head()

,"[D210000] Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW)","[D210000] Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW).1","[D210000] Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW).2","[D210000] Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW).3","[D210000] Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW).4","[D210000] Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW).5","[D210000] Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW).6",20191231,20181231,20171231
0,concept_id,label_ko,label_en,class0,class1,class2,class3,"('연결재무제표',)","('연결재무제표',)","('연결재무제표',)"
1,ifrs-full_CurrentAssets,유동자산,Current assets,재무상태표 [abstract],자산 [abstract],유동자산,NaN,13364642030.0,14058952699.0,15367546611.0
2,ifrs-full_CashAndCashEquivalents,현금및현금성자산,Cash and cash equivalents,재무상태표 [abstract],자산 [abstract],유동자산,현금및현금성자산,804895857.0,2514414223.0,285444330.0
3,ifrs-full_TradeAndOtherCurrentReceivables,매출채권 및 기타채권,Trade and other current receivables,재무상태표 [abstract],자산 [abstract],유동자산,매출채권 및 기타채권,3940691618.0,2890427236.0,5904658474.0
4,dart_OtherCurrentAssets,기타유동자산,Other current assets,재무상태표 [abstract],자산 [abstract],유동자산,기타유동자산,1390777847.0,1774596881.0,1496437587.0


In [13]:
try:
    if df_bs.columns[6].isdigit() == True:
        df_bs_1 = df_bs.iloc[:, [1, 6, 7, 8]]
    elif df_bs.columns[7].isdigit() == True:
        df_bs_1 = df_bs.iloc[:, [1, 7, 8, 9]]
    else:
        df_bs_1 = df_bs.iloc[:, [1, 8, 9, 10]]
except:
    df_bs_1 = df_bs.iloc[:, 0:4]

In [14]:
df_bs_1.columns = ['', df_bs_1.columns[1], df_bs_1.columns[2], df_bs_1.columns[3]]

In [15]:
df_bs_1 = df_bs_1.T

In [16]:
df_bs_1

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
,label_ko,유동자산,현금및현금성자산,매출채권 및 기타채권,기타유동자산,당기법인세자산,재고자산,비유동자산,매출채권 및 기타채권,기타금융상품,...,비유동리스부채,부채총계,지배기업의 소유주에게 귀속되는 자본,자본금,자본잉여금,기타자본구성요소,결손금,비지배지분,자본총계,자본과부채총계
20191231,"('연결재무제표',)",13364642030.0,804895857.0,3940691618.0,1390777847.0,3902370.0,7224374338.0,8642675058.0,1811108544.0,473200000.0,...,1332002356.0,12176675230.0,9846112452.0,8233065500.0,21921522495.0,-741545546.0,-19566929997.0,-15470594.0,9830641858.0,22007317088.0
20181231,"('연결재무제표',)",14058952699.0,2514414223.0,2890427236.0,1774596881.0,5159740.0,6874354619.0,6537461785.0,1694820785.0,120000000.0,...,0.0,5779552952.0,14816861532.0,8233065500.0,21921522495.0,-757214630.0,-14580511833.0,0.0,14816861532.0,20596414484.0
20171231,"('연결재무제표',)",15367546611.0,285444330.0,5904658474.0,1496437587.0,14319300.0,7666686920.0,12541733326.0,756300785.0,1166214776.0,...,0.0,10740876230.0,17168403707.0,8233065500.0,21921522495.0,-804323422.0,-12181860866.0,0.0,17168403707.0,27909279937.0


In [17]:
df_bs_1 = df_bs_1.rename(columns=df_bs_1.iloc[0])

In [26]:
df_bs_1

,date,유동자산,현금및현금성자산,매출채권 및 기타채권,기타유동자산,당기법인세자산,재고자산,비유동자산,매출채권 및 기타채권,기타금융상품,...,부채총계,지배기업의 소유주에게 귀속되는 자본,자본금,자본잉여금,기타자본구성요소,결손금,비지배지분,자본총계,자본과부채총계,company
0,20191231,13364642030.0,804895857.0,3940691618.0,1390777847.0,3902370.0,7224374338.0,8642675058.0,1811108544.0,473200000.0,...,12176675230.0,9846112452.0,8233065500.0,21921522495.0,-741545546.0,-19566929997.0,-15470594.0,9830641858.0,22007317088.0,ITX엠투엠
1,20181231,14058952699.0,2514414223.0,2890427236.0,1774596881.0,5159740.0,6874354619.0,6537461785.0,1694820785.0,120000000.0,...,5779552952.0,14816861532.0,8233065500.0,21921522495.0,-757214630.0,-14580511833.0,0.0,14816861532.0,20596414484.0,ITX엠투엠
2,20171231,15367546611.0,285444330.0,5904658474.0,1496437587.0,14319300.0,7666686920.0,12541733326.0,756300785.0,1166214776.0,...,10740876230.0,17168403707.0,8233065500.0,21921522495.0,-804323422.0,-12181860866.0,0.0,17168403707.0,27909279937.0,ITX엠투엠


In [19]:
df_bs_1 = df_bs_1.drop(df_bs_1.index[0])

In [21]:
df_bs_1.drop('label_ko', inplace=True, axis =1)

In [23]:
df_bs_1 = df_bs_1.reset_index().rename(columns={'index': 'date'})

In [25]:
df_bs_1['company'] = file_list1[3][:-13]

In [27]:
print('bs_preprocessing complete')

bs_preprocessing complete


In [28]:
var_1_dic = {'자본총계': 4, '자산총계': 4, '매입채무': 11, '매출채권': 11,
         '부채총계': 4, '유동부채': 4, '유동자산': 4, '재고자산': 4}
var_2_dic = {'매출총이익': 5, '법인세비용차감': 15, '당기순이익': 10}

In [29]:
bs_a = pd.DataFrame(columns=['date', 'company'])

In [30]:
list1 = ['date', 'company']
for var1, var1_ in var_1_dic.items():
    for bs in df_bs_1.columns:
        if var1 in bs:
            if var1_ >= len(bs):
                list1.append(bs)
print(df_bs_1[list1])

       date company           자본총계           자산총계   매입채무 및 기타채무   매입채무 및 기타채무  \
0  20191231  ITX엠투엠   9830641858.0  22007317088.0  3716596069.0   120000000.0   
1  20181231  ITX엠투엠  14816861532.0  20596414484.0  2302821876.0           0.0   
2  20171231  ITX엠투엠  17168403707.0  27909279937.0  3694364681.0  1137142861.0   

    매입채무 및 기타채무   매입채무 및 기타채무   매출채권 및 기타채권   매출채권 및 기타채권   매출채권 및 기타채권  \
0  3716596069.0   120000000.0  3940691618.0  1811108544.0  3940691618.0   
1  2302821876.0           0.0  2890427236.0  1694820785.0  2890427236.0   
2  3694364681.0  1137142861.0  5904658474.0   756300785.0  5904658474.0   

    매출채권 및 기타채권           부채총계          유동부채           유동자산          재고자산  
0  1811108544.0  12176675230.0  9899998168.0  13364642030.0  7224374338.0  
1  1694820785.0   5779552952.0  5521874223.0  14058952699.0  6874354619.0  
2   756300785.0  10740876230.0  9603733369.0  15367546611.0  7666686920.0  


In [36]:
list1

['date',
 'company',
 '자본총계',
 '자산총계',
 '매입채무 및 기타채무',
 '매입채무 및 기타채무',
 '매출채권 및 기타채권',
 '매출채권 및 기타채권',
 '부채총계',
 '유동부채',
 '유동자산',
 '재고자산']

In [32]:
df_bs_1[list1]

,date,company,자본총계,자산총계,매입채무 및 기타채무,매입채무 및 기타채무,매입채무 및 기타채무,매입채무 및 기타채무,매출채권 및 기타채권,매출채권 및 기타채권,매출채권 및 기타채권,매출채권 및 기타채권,부채총계,유동부채,유동자산,재고자산
0,20191231,ITX엠투엠,9830641858.0,22007317088.0,3716596069.0,120000000.0,3716596069.0,120000000.0,3940691618.0,1811108544.0,3940691618.0,1811108544.0,12176675230.0,9899998168.0,13364642030.0,7224374338.0
1,20181231,ITX엠투엠,14816861532.0,20596414484.0,2302821876.0,0.0,2302821876.0,0.0,2890427236.0,1694820785.0,2890427236.0,1694820785.0,5779552952.0,5521874223.0,14058952699.0,6874354619.0
2,20171231,ITX엠투엠,17168403707.0,27909279937.0,3694364681.0,1137142861.0,3694364681.0,1137142861.0,5904658474.0,756300785.0,5904658474.0,756300785.0,10740876230.0,9603733369.0,15367546611.0,7666686920.0


In [34]:
alpha = pd.concat([df_bs_1[list1]])

In [35]:
alpha

,date,company,자본총계,자산총계,매입채무 및 기타채무,매입채무 및 기타채무,매입채무 및 기타채무,매입채무 및 기타채무,매출채권 및 기타채권,매출채권 및 기타채권,매출채권 및 기타채권,매출채권 및 기타채권,부채총계,유동부채,유동자산,재고자산
0,20191231,ITX엠투엠,9830641858.0,22007317088.0,3716596069.0,120000000.0,3716596069.0,120000000.0,3940691618.0,1811108544.0,3940691618.0,1811108544.0,12176675230.0,9899998168.0,13364642030.0,7224374338.0
1,20181231,ITX엠투엠,14816861532.0,20596414484.0,2302821876.0,0.0,2302821876.0,0.0,2890427236.0,1694820785.0,2890427236.0,1694820785.0,5779552952.0,5521874223.0,14058952699.0,6874354619.0
2,20171231,ITX엠투엠,17168403707.0,27909279937.0,3694364681.0,1137142861.0,3694364681.0,1137142861.0,5904658474.0,756300785.0,5904658474.0,756300785.0,10740876230.0,9603733369.0,15367546611.0,7666686920.0


In [ ]:
date	company	자본총계	자산총계	단기매입채무	부채총계	유동부채	유동자산	재고자산	장기매입채무및기타채무	매출채권

In [55]:
df_cis_1

,date,label_ko,수익,국내공사수입,해외공사수입,임대수입,제품매출수입,기타매출수입,분양매출수입,매출원가,...,기타포괄손익,해외사업환산손익,매도가능금융자산평가손익,보험수리적손익,총포괄손익,"포괄손익, 지배기업의 소유주에게 귀속되는 지분","포괄손익, 비지배지분",기본주당이익(손실),희석주당이익(손실),company
0,0,20110101-20111231,수익,국내공사수입,해외공사수입,임대수입,제품매출수입,기타매출수입,분양매출수입,매출원가,...,기타포괄손익,해외사업환산손익,매도가능금융자산평가손익,보험수리적손익,총포괄손익,"포괄손익, 지배기업의 소유주에게 귀속되는 지분","포괄손익, 비지배지분",기본주당이익(손실),희석주당이익(손실),풍림산업
1,1,20100101-20101231,1040133868537.0,998594426801.0,5139959955.0,5044777566.0,16346058014.0,14988123813.0,20522388.0,1038061167643.0,...,3540805607.0,2240108630.0,1786556576.0,-485859599.0,-32311758157.0,-32311758157.0,NaN,-2021.0,-2021.0,풍림산업
2,2,"('연결재무제표',)",1115226030432.0,1064823068372.0,10889326571.0,4976794251.0,23829611248.0,10707229990.0,0.0,1086181866365.0,...,-1470609662.0,-628693573.0,-844092501.0,2176412.0,-76094685747.0,-76094685747.0,NaN,-4207.0,-4207.0,풍림산업


In [12]:
var_1_dic = {'자본총계': 4, '자산총계': 4, '매입채무': 11, '매출채권': 11,
         '부채총계': 4, '유동부채': 4, '유동자산': 4, '재고자산': 4, 'company': 7}
var_2_dic = {'date':4, '매출총이익': 5, '법인세비용차감': 15, '당기순이익': 10,
         'company': 7}

cis_a = pd.DataFrame(0, index= np.arange(15), columns=['date', '매출총이익', '법인세비용차감전순이익', '당기순이익'])

file_list1 = os.listdir(FOLDER_PATH +'재무상태표/')
file_list2 = os.listdir(FOLDER_PATH +'포괄손익계산서/')

count = 0

In [9]:
file_list2

['AP시스템 포괄손익계산서 원본.csv',
 'DMS 포괄손익계산서 원본.csv',
 'HRS 포괄손익계산서 원본.csv',
 'ITX엠투엠 포괄손익계산서 원본.csv',
 'SGA 포괄손익계산서 원본.csv',
 'SM Life Design 포괄손익계산서 원본.csv',
 '노바렉스 포괄손익계산서 원본.csv',
 '대성엘텍 포괄손익계산서 원본.csv',
 '대성파인텍 포괄손익계산서 원본.csv',
 '대한과학 포괄손익계산서 원본.csv',
 '대한그린파워 포괄손익계산서 원본.csv',
 '동국산업 포괄손익계산서 원본.csv',
 '로보스타 포괄손익계산서 원본.csv',
 '로체시스템즈 포괄손익계산서 원본.csv',
 '메디톡스 포괄손익계산서 원본.csv',
 '멜파스 포괄손익계산서 원본.csv',
 '명성티엔에스 포괄손익계산서 원본.csv',
 '미래생명자원 포괄손익계산서 원본.csv',
 '미코 포괄손익계산서 원본.csv',
 '바른테크놀로지 포괄손익계산서 원본.csv',
 '바이오스마트 포괄손익계산서 원본.csv',
 '베셀 포괄손익계산서 원본.csv',
 '비아트론 포괄손익계산서 원본.csv',
 '삼강엠앤티 포괄손익계산서 원본.csv',
 '상상인인더스트리 포괄손익계산서 원본.csv',
 '서플러스글로벌 포괄손익계산서 원본.csv',
 '성도이엔지 포괄손익계산서 원본.csv',
 '세경하이테크 포괄손익계산서 원본.csv',
 '세운메디칼 포괄손익계산서 원본.csv',
 '슈펙스비앤피 포괄손익계산서 원본.csv',
 '스킨앤스킨 포괄손익계산서 원본.csv',
 '아바코 포괄손익계산서 원본.csv',
 '아스트 포괄손익계산서 원본.csv',
 '아이씨디 포괄손익계산서 원본.csv',
 '아이원스 포괄손익계산서 원본.csv',
 '에스에프에이 포괄손익계산서 원본.csv',
 '에이프런티어 포괄손익계산서 원본.csv',
 '엔피케이 포괄손익계산서 원본.csv',
 '엘오티베큠 포괄손익계산서 원본.csv',
 '예스티 포괄손익계산서 원본.csv',
 '오르

In [10]:
cis_a

,date,company,매출총이익,법인세비용차감전순이익(손실),당기순이익(손실),법인세비용차감전순이익,당기순이익,당기순이익의 귀속
0,20191231,AP시스템,72622039976.0,12402291609.0,9451469310.0,NaN,NaN,NaN
1,20181231,AP시스템,101171251514.0,27943908986.0,25227011908.0,NaN,NaN,NaN
2,20171231,AP시스템,62076108001.0,26316191703.0,19088064733.0,NaN,NaN,NaN
0,20191231,DMS,65300086704.0,21544765028.0,17236232679.0,NaN,NaN,NaN
1,20181231,DMS,75967702026.0,21868026243.0,15184128518.0,NaN,NaN,NaN
2,20171231,DMS,61208001998.0,4835649098.0,3857602202.0,NaN,NaN,NaN
0,20191231,HRS,9322633707.0,NaN,NaN,1092375329.0,982759631.0,NaN
1,20181231,HRS,12530099460.0,NaN,NaN,-9625003170.0,-7481797843.0,NaN
2,20171231,HRS,15194350699.0,NaN,NaN,7943504959.0,6287854808.0,NaN


In [15]:
df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + '포괄손익계산서/' + file_list2[0]))
df_cis_1 = company_prerocessing.cis_preprocessing(df, cmp_list2[:-15])

for var2, var2_ in var_2_dic.items():
    for cis in df_cis_1.columns:
        if var2 in cis:
            if var2_ >= len(cis):
                print(cis)
                cis_a.iloc[0][count] = df_cis_1[cis]
                count += 1

NameError: name 'cmp_list2' is not defined

In [16]:
for cmp_list2 in file_list2:
    df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + '포괄손익계산서/' + cmp_list2))
    df_cis_1 = company_prerocessing.cis_preprocessing(df, cmp_list2[:-15])
        
    for var2, var2_ in var_2_dic.items():
        for cis in df_cis_1.columns:
            if var2 in cis:
                if var2_ >= len(cis):
                    print(cis)
                    cis_a.iloc[:, count] = df_cis_1[cis]
                    count += 1

KeyError: "['level_0'] not found in axis"

In [56]:
os.path.join(FOLDER_PATH , file_list[0])

NameError: name 'file_list' is not defined

In [ ]:
file_list[0]

In [119]:
df_cis = company_prerocessing.open_df(os.path.join(FOLDER_PATH +'포괄손익계산서/'+file_list2[1]))

In [120]:
df_cis.head()

,"[D431410] Statement of comprehensive income, by function of expense - Consolidated financial statements (Unit: KRW)","[D431410] Statement of comprehensive income, by function of expense - Consolidated financial statements (Unit: KRW).1","[D431410] Statement of comprehensive income, by function of expense - Consolidated financial statements (Unit: KRW).2","[D431410] Statement of comprehensive income, by function of expense - Consolidated financial statements (Unit: KRW).3","[D431410] Statement of comprehensive income, by function of expense - Consolidated financial statements (Unit: KRW).4","[D431410] Statement of comprehensive income, by function of expense - Consolidated financial statements (Unit: KRW).5",20110101-20111231,20100101-20101231
0,concept_id,label_ko,label_en,class0,class1,class2,"('연결재무제표',)","('연결재무제표',)"
1,ifrs_Revenue,수익,Revenue(Sales),포괄손익계산서 [abstract],수익,NaN,81567060964.0,102325981917.0
2,ifrs_CostOfSales,매출원가,Cost of sales,포괄손익계산서 [abstract],매출원가,NaN,73056723918.0,94625046499.0
3,ifrs_GrossProfit,매출총이익,Gross profit,포괄손익계산서 [abstract],매출총이익,NaN,8510337046.0,7700935418.0
4,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,Selling general administrative expenses ...,포괄손익계산서 [abstract],판매비와관리비,NaN,5065406425.0,9233609836.0


In [135]:
if '01-20' in df_cis.columns[6]:
    df_cis_1 = df_cis.iloc[:, [1, 6, 7]]
else:
    df_cis_1 = df_cis.iloc[:, [1, 7, 8]]


df_cis_1.columns

df_cis_1.columns[1]

df_cis_1.columns = ['', df_cis_1.columns[1], df_cis_1.columns[2]]
df_cis_1

df_cis_1 = df_cis_1.T

df_cis_1

df_cis_1 = df_cis_1.rename(columns=df_cis_1.iloc[0])

df_cis_1

df_cis_1 = df_cis_1.drop(df_cis_1.index[0])

df_cis_1

df_cis_1.drop('label_ko', inplace=True, axis =1)

df_cis_1

df_cis_1

,수익,매출원가,매출총이익,판매비와관리비,영업이익(손실),기타수익,기타비용,금융수익,금융원가,법인세비용차감전순이익(손실),...,당기순이익(손실),지배기업의 소유주에게 귀속되는 당기순이익(손실),비지배지분에 귀속되는 당기순이익(손실),연결기타포괄손익,해외사업환산손익,보험수리적손익,연결당기총포괄이익,"포괄손익, 지배기업의 소유주에게 귀속되는 지분","포괄손익, 비지배지분",기본주당이익(손실)
20110101-20111231,81567060964.0,73056723918.0,8510337046.0,5065406425.0,3444930621.0,3330603897.0,2058192368.0,198325041.0,4539426603.0,376240588.0,...,627781200.0,627781200.0,0.0,1720221602.0,1769414358.0,-49192756.0,2348002802.0,2348002802.0,0.0,7.0
20100101-20101231,102325981917.0,94625046499.0,7700935418.0,9233609836.0,-1532674418.0,1857924763.0,1751015242.0,100126194.0,4156786100.0,-5482424803.0,...,-5245282081.0,-5245282081.0,0.0,378372976.0,456954301.0,-78581325.0,-4866909105.0,-4866909105.0,0.0,-79.0


In [ ]:
        df_cis_1 = df_cis_1.reset_index().rename(columns={'index': 'date'})
        # df_cis_1['date'] = df_cis_1['date'].str[9:]
#         df_cis_1.drop(['level_0', 'label_ko'], axis=1, inplace=True)

        df_cis_1['company'] = cmp_list
        print('cis_preprocessing complete')

In [ ]:
'01-20' in df_cis.columns[6]

In [ ]:
df_cis.columns[6].isdigit() == True

In [ ]:
df_cis.head()

In [ ]:
df_cis_1 = df_cis.iloc[:, [1, 7,8]]

In [ ]:
df_cis_1.head()

In [ ]:
df_cis_1.columns = ['', df_cis_1.columns[1][0], df_cis_1.columns[2][0]]

In [ ]:
df_cis_1 = df_cis_1.T

In [ ]:
df_cis_1.columns = df_cis_1.columns.astype('object')

In [ ]:
for i in range(len(df_cis_1.columns)):
            df_cis_1.columns.values[i] = df_cis_1.iloc[0, :][i]

In [ ]:
df_cis_1.reset_index(inplace=True)
df_cis_1.drop(0, inplace=True)

In [ ]:
for i in range(2):
    df_cis_1.iloc[i, 0] = df_cis.columns.values[i+6]

In [ ]:
df_cis_1 = df_cis_1.reset_index().rename(columns={'index': 'date'})
df_cis_1.drop(['level_0', 'label_ko'], axis=1, inplace=True)

In [ ]:
def cis_preprocessing(df_cis):
    df_cis_1 = df_cis.iloc[:, [1, 7, 8, 9]]
    df_cis_1.columns = ['', df_cis_1.columns[1][0], df_cis_1.columns[2][0], df_cis_1.columns[3][0]]
    df_cis_1 = df_cis_1.T
    df_cis_1 = df_cis_1.rename(columns=df_cis_1.iloc[0])
    df_cis_1 = df_cis_1.drop(df_cis_1.index[0])
    df_cis_1 = df_cis_1.reset_index().rename(columns={'index': 'date'})
    df_cis_1['date'] = df_cis_1['date'].str[9:]
    print('cis_proprocessing complete')

    return df_cis_1

In [ ]:
file_list[4]

In [ ]:
df = company_prerocessing.open_df(os.path.join(FOLDER_PATH , file_list[8]))
print(file_list[8])
df.head()

In [ ]:
df_bs_1 = df.iloc[:, 0:4]

In [ ]:
df_bs_1.head()

In [ ]:
df_bs_1.columns = ['', df_bs_1.columns[1][0], df_bs_1.columns[2][0], df_bs_1.columns[3][0]]

In [ ]:
df_bs_1

In [ ]:
df_bs_1 = df_bs_1.T

In [ ]:
df_bs_1.head()

In [ ]:
df_bs_1.columns = df_bs_1.columns.astype('object')

In [ ]:
for i in range(len(df_bs_1.columns)):
    
    df_bs_1.columns.values[i] = df_bs_1.iloc[0, :][i]

In [ ]:
df_bs_1.reset_index(inplace=True)

In [ ]:
df_bs_1

In [ ]:
df_bs_1.drop(0, inplace=True)

In [ ]:
df_bs_1.head()

In [ ]:
df_bs_1.iloc[1, 0]

In [ ]:
df.columns.values[1]

In [ ]:
df_bs_1.iloc[0, 0] = df.columns.values[1]
df_bs_1.iloc[1, 0] = df.columns.values[2]
df_bs_1.iloc[2, 0] = df.columns.values[3]

In [ ]:
df_bs_1

In [ ]:
# df_bs_1 = df_bs_1.rename(columns=df_bs_1.iloc[0])
# df_bs_1 = df_bs_1.drop(df_bs_1.index[0])
df_bs_1 = df_bs_1.reset_index().rename(columns={'index': 'date'})

In [ ]:
df_bs_1.drop(['level_0', 'label_ko'], axis=1, inplace=True)

In [ ]:
df_bs_1